In [218]:
# Standard library imports
import os
import sys
import re
import warnings
import random
import hashlib

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning and preprocessing
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler  # Assuming you might need it

# Specific models and tools
from xgboost import XGBClassifier
import xgboost as xgb

# Encoding and feature selection
from category_encoders import TargetEncoder  # Fixed the import based on usage
from scipy.stats import randint, uniform

# Model persistence
from joblib import dump, load

# Miscellaneous settings
%matplotlib inline
warnings.filterwarnings('ignore')


In [219]:
comps = [
    'E0', 
    'E1', 
    
    #'E2', 'E3',    
    #'SC0', 'SC1',

    'D1', 
    'D2',
    'F1', 
    'F2',
    'I1', 
    'I2',
    'SP1', 
    'SP2',
    #'B1',
    #'G1',
    #'N1',
    #'P1',
    #'T1',
]

seasons = [
    '2324', 
    '2223', '2122', '2021',
    '1920', 
    #'1819', '1718', '1617',
    #'1516', '1415', '1314', '1213',
    #'1112', '1011', 
    #'0910', '0809',
    #'0708', '0607', '0506', '0405',
    #'0304', '0203', '0102', '0001',
]

countries = [
    "ARG", "AUT", "BRA", "CHN",
    "DNK", "FIN", "IRL", "JPN",
    "MEX", "NOR", "POL", "ROU",
    "RUS", "SWE", "SWZ", "USA",
]

fixtures = [
    "fixtures",
    #"new_league_fixtures"
]

content = "big5_1en2_fixtures"

In [220]:
# Load all filepaths into a list
matches_files = []

In [221]:
for season in seasons:    
    for comp in comps:  
        matches_files.append('data/scraped/%s/%s.csv' % (season, comp))
        print()  # dummy line in case the line above is commented out

In [222]:
for country in countries:    
    #matches_files.append('data/scraped/other/%s.csv' % (country))
    print()  # dummy line in case the line above is commented out

In [223]:
for fixture in fixtures:    
    matches_files.append('data/scraped/fixtures/%s.csv' % (fixture))
    print()  # dummy line in case the line above is commented out

In [224]:
matches_files

['data/scraped/2324/E0.csv',
 'data/scraped/2324/E1.csv',
 'data/scraped/2324/D1.csv',
 'data/scraped/2324/D2.csv',
 'data/scraped/2324/F1.csv',
 'data/scraped/2324/F2.csv',
 'data/scraped/2324/I1.csv',
 'data/scraped/2324/I2.csv',
 'data/scraped/2324/SP1.csv',
 'data/scraped/2324/SP2.csv',
 'data/scraped/2223/E0.csv',
 'data/scraped/2223/E1.csv',
 'data/scraped/2223/D1.csv',
 'data/scraped/2223/D2.csv',
 'data/scraped/2223/F1.csv',
 'data/scraped/2223/F2.csv',
 'data/scraped/2223/I1.csv',
 'data/scraped/2223/I2.csv',
 'data/scraped/2223/SP1.csv',
 'data/scraped/2223/SP2.csv',
 'data/scraped/2122/E0.csv',
 'data/scraped/2122/E1.csv',
 'data/scraped/2122/D1.csv',
 'data/scraped/2122/D2.csv',
 'data/scraped/2122/F1.csv',
 'data/scraped/2122/F2.csv',
 'data/scraped/2122/I1.csv',
 'data/scraped/2122/I2.csv',
 'data/scraped/2122/SP1.csv',
 'data/scraped/2122/SP2.csv',
 'data/scraped/2021/E0.csv',
 'data/scraped/2021/E1.csv',
 'data/scraped/2021/D1.csv',
 'data/scraped/2021/D2.csv',
 'data/s

In [225]:
# Load and concatenate matches data into a single DataFrame
df = pd.DataFrame()

for file in matches_files:

    try:
        df_temp = pd.read_csv(file)
        df = pd.concat([df, df_temp], ignore_index=True)
    except:
        # print an error message
        print(f'Error: {file} not found')

# print the amount of data loaded
print(f"Data loaded: {df.shape[0]} matches")

Data loaded: 18633 matches


In [226]:
# print every column in the DataFrame, along with its data type, line by line
for column in df.columns:
    print(column)

Div
Date
Time
HomeTeam
AwayTeam
FTHG
FTAG
FTR
HTHG
HTAG
HTR
Referee
HS
AS
HST
AST
HF
AF
HC
AC
HY
AY
HR
AR
B365H
B365D
B365A
BWH
BWD
BWA
IWH
IWD
IWA
PSH
PSD
PSA
WHH
WHD
WHA
VCH
VCD
VCA
MaxH
MaxD
MaxA
AvgH
AvgD
AvgA
B365>2.5
B365<2.5
P>2.5
P<2.5
Max>2.5
Max<2.5
Avg>2.5
Avg<2.5
AHh
B365AHH
B365AHA
PAHH
PAHA
MaxAHH
MaxAHA
AvgAHH
AvgAHA
B365CH
B365CD
B365CA
BWCH
BWCD
BWCA
IWCH
IWCD
IWCA
PSCH
PSCD
PSCA
WHCH
WHCD
WHCA
VCCH
VCCD
VCCA
MaxCH
MaxCD
MaxCA
AvgCH
AvgCD
AvgCA
B365C>2.5
B365C<2.5
PC>2.5
PC<2.5
MaxC>2.5
MaxC<2.5
AvgC>2.5
AvgC<2.5
AHCh
B365CAHH
B365CAHA
PCAHH
PCAHA
MaxCAHH
MaxCAHA
AvgCAHH
AvgCAHA


In [227]:
# Rename columns if they exist
df.rename(columns={
    'Country': 'Div',
    'Home': 'HomeTeam',
    'Away': 'AwayTeam',
    'Res': 'FTR',

}, inplace=True)

In [228]:
# Remove all rows where 'Div' is not in the list of competitions
df = df[df['Div'].isin(comps)].copy()

In [229]:
df.tail(20)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
18585,SP2,20/07/2020,20:00,Cadiz,Albacete,0.0,1.0,A,0.0,0.0,...,1.54,0.00,2.13,1.75,2.15,1.76,2.19,1.81,2.10,1.76
18586,SP2,20/07/2020,20:00,Elche,Oviedo,2.0,1.0,H,1.0,1.0,...,1.73,-1.00,1.90,1.95,1.97,1.93,2.03,1.97,1.92,1.91
18587,SP2,20/07/2020,20:00,Lugo,Mirandes,2.0,1.0,H,0.0,1.0,...,1.80,-0.50,1.93,1.93,1.94,1.96,2.01,2.07,1.87,1.96
18588,SP2,20/07/2020,20:00,Numancia,Tenerife,2.0,1.0,H,1.0,0.0,...,1.90,-0.75,1.83,2.02,1.87,2.03,1.88,2.12,1.83,2.02
18589,SP2,20/07/2020,20:00,Santander,Vallecano,1.0,2.0,A,1.0,0.0,...,2.18,1.00,2.02,1.83,2.06,1.84,2.11,1.87,2.03,1.81
18590,SP2,20/07/2020,20:00,Sp Gijon,Huesca,0.0,1.0,A,0.0,0.0,...,1.87,0.00,2.02,1.83,2.02,1.88,2.18,1.90,2.02,1.83
18591,SP2,20/07/2020,20:00,Zaragoza,Ponferradina,2.0,1.0,H,1.0,0.0,...,1.93,0.00,1.70,2.10,1.72,2.21,1.87,2.21,1.74,2.12
18592,SP2,07/08/2020,20:00,La Coruna,Fuenlabrada,2.0,1.0,H,0.0,1.0,...,1.83,0.00,1.85,2.00,1.88,1.97,2.15,2.08,1.87,1.98
18593,E1,09/04/2024,19:45,Millwall,Leicester,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18594,E1,09/04/2024,19:45,Plymouth,QPR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [230]:
# Check for duplicate column names
print(df.columns[df.columns.duplicated()])


Index([], dtype='object')


In [231]:
# Convert 'Div' to a categorical type, a numeric representation of the division
df['Div'] = df['Div'].astype('category').cat.codes

In [232]:
# Assuming 'teams' is a list of team names
teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).dropna().unique()
teams.sort()

# Creating a dictionary from team names to an incremental index number
teams_dict = {team: index for index, team in enumerate(teams)}

In [233]:
# Save the dictionary to a file
with open(f'data/teams_dict_{content}.txt', 'w') as file:
    file.write(str(teams_dict))

In [234]:
df.tail(20)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
18585,9,20/07/2020,20:00,Cadiz,Albacete,0.0,1.0,A,0.0,0.0,...,1.54,0.00,2.13,1.75,2.15,1.76,2.19,1.81,2.10,1.76
18586,9,20/07/2020,20:00,Elche,Oviedo,2.0,1.0,H,1.0,1.0,...,1.73,-1.00,1.90,1.95,1.97,1.93,2.03,1.97,1.92,1.91
18587,9,20/07/2020,20:00,Lugo,Mirandes,2.0,1.0,H,0.0,1.0,...,1.80,-0.50,1.93,1.93,1.94,1.96,2.01,2.07,1.87,1.96
18588,9,20/07/2020,20:00,Numancia,Tenerife,2.0,1.0,H,1.0,0.0,...,1.90,-0.75,1.83,2.02,1.87,2.03,1.88,2.12,1.83,2.02
18589,9,20/07/2020,20:00,Santander,Vallecano,1.0,2.0,A,1.0,0.0,...,2.18,1.00,2.02,1.83,2.06,1.84,2.11,1.87,2.03,1.81
18590,9,20/07/2020,20:00,Sp Gijon,Huesca,0.0,1.0,A,0.0,0.0,...,1.87,0.00,2.02,1.83,2.02,1.88,2.18,1.90,2.02,1.83
18591,9,20/07/2020,20:00,Zaragoza,Ponferradina,2.0,1.0,H,1.0,0.0,...,1.93,0.00,1.70,2.10,1.72,2.21,1.87,2.21,1.74,2.12
18592,9,07/08/2020,20:00,La Coruna,Fuenlabrada,2.0,1.0,H,0.0,1.0,...,1.83,0.00,1.85,2.00,1.88,1.97,2.15,2.08,1.87,1.98
18593,3,09/04/2024,19:45,Millwall,Leicester,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18594,3,09/04/2024,19:45,Plymouth,QPR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [235]:
# Create a unique list of HomeTeam and AwayTeam names combined, and add an index to each team
teams = pd.concat([df['HomeTeam'], df['AwayTeam']]).dropna().unique()

# Sort the teams alphabetically
teams.sort()

# Convert to an array of dictionaries
teams = [{'team': team, 'index': index} for index, team in enumerate(teams)]

df['Team_ID'] = df['HomeTeam'].map(teams_dict)
df['Opp_ID'] = df['AwayTeam'].map(teams_dict)

In [236]:
# if the column 'Referee' exists, convert it to a categorical type
if 'Referee' in df.columns:
    # Create a unique list of Referees, and add an index to each Referee
    referees = pd.concat([df['Referee']]).unique()

    # Convert to an array of dictionaries
    referees = [{'referee': referee, 'index': index} for index, referee in enumerate(referees)]

else:
    df['Referee'] = 0

In [237]:
# Map the team names to the index values in the 'teams' list
#df['Team_ID'] = df['HomeTeam'].map({team['team']: team['index'] for team in teams})
#df['Opp_ID'] = df['AwayTeam'].map({team['team']: team['index'] for team in teams})

In [238]:
# Correcting the creation of a unique list of Referees and adding an index to each Referee
referees = df['Referee'].unique()  # This should directly refer to the 'Referee' column

if len(referees) > 0:
    # Convert to a dictionary with referee names as keys and their indices as values
    referee_dict = {referee: index for index, referee in enumerate(referees)}

    # Now map the 'Referee' column to these indices
    df['Ref_ID'] = df['Referee'].map(referee_dict)
else:
    # If there are no referees, create a dummy column with all zeros
    df['Ref_ID'] = 0

In [239]:
import pandas as pd

def parse_date_to_int(date_str):
    # Split the date_str by the "/" character into day, month, year
    components = date_str.split('/')
    
    # If split was successful but not in expected format, try splitting by absence of separator for '%d%m%Y' or '%d%m%y'
    if len(components) == 1:
        if len(date_str) in [6, 8]:  # Length 6 for '%d%m%y', 8 for '%d%m%Y'
            day, month = int(date_str[:2]), int(date_str[2:4])
            year = int(date_str[4:])
        else:
            return 19000101  # Return default if format does not match expected
    else:
        day, month = int(components[0]), int(components[1])
        year = int(components[2])
    
    # Adjust the year if it was only 2 characters long
    if year < 100:
        year += 2000
    
    # Create a date variable by using the day, month, year integers
    # Note: Direct creation of date variable skipped to avoid unnecessary complexity,
    # directly formatting to YYYYMMDD integer format instead.
    date_int = int(f"{year:04d}{month:02d}{day:02d}")
    
    return date_int

# Assuming 'df' is your DataFrame and 'Date' is the column you want to convert
# First, ensure the Date column is in a datetime format if it's not already
df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)

# Apply the modified function
df['Date_temp'] = df['Date'].apply(lambda x: parse_date_to_int(x.strftime('%d/%m/%Y')) if pd.notnull(x) else 19000101)


In [240]:
df.tail(20)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Team_ID,Opp_ID,Ref_ID,Date_temp
18585,9,2020-07-20,20:00,Cadiz,Albacete,0.0,1.0,A,0.0,0.0,...,2.15,1.76,2.19,1.81,2.10,1.76,42,2,54,20200720
18586,9,2020-07-20,20:00,Elche,Oviedo,2.0,1.0,H,1.0,1.0,...,1.97,1.93,2.03,1.97,1.92,1.91,72,167,54,20200720
18587,9,2020-07-20,20:00,Lugo,Mirandes,2.0,1.0,H,0.0,1.0,...,1.94,1.96,2.01,2.07,1.87,1.96,133,148,54,20200720
18588,9,2020-07-20,20:00,Numancia,Tenerife,2.0,1.0,H,1.0,0.0,...,1.87,2.03,1.88,2.12,1.83,2.02,162,220,54,20200720
18589,9,2020-07-20,20:00,Santander,Vallecano,1.0,2.0,A,1.0,0.0,...,2.06,1.84,2.11,1.87,2.03,1.81,199,232,54,20200720
18590,9,2020-07-20,20:00,Sp Gijon,Huesca,0.0,1.0,A,0.0,0.0,...,2.02,1.88,2.18,1.90,2.02,1.83,209,104,54,20200720
18591,9,2020-07-20,20:00,Zaragoza,Ponferradina,2.0,1.0,H,1.0,0.0,...,1.72,2.21,1.87,2.21,1.74,2.12,249,179,54,20200720
18592,9,2020-08-07,20:00,La Coruna,Fuenlabrada,2.0,1.0,H,0.0,1.0,...,1.88,1.97,2.15,2.08,1.87,1.98,114,87,54,20200807
18593,3,2024-04-09,19:45,Millwall,Leicester,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,147,124,54,20240409
18594,3,2024-04-09,19:45,Plymouth,QPR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,178,182,54,20240409


In [241]:
df['Time'] = df['Time'].fillna('00:00').str.replace(':', '').astype(int)

In [242]:
df.columns = [re.sub(r'[<]', '_st_', str(col)) for col in df.columns]
df.columns = [re.sub(r'[>]', '_gt_', str(col)) for col in df.columns]

In [243]:
# Sort df by Date_temp and Time
df = df.sort_values(['Date_temp', 'Time'])

In [244]:
def history_vs_opponent_weighted(df, row, team_column):
    # Initialize the total weighted score
    weighted_score = 0
    opponent_column = 'Opp_ID'

    row_date_temp = row['Date'].year * 10000 + row['Date'].month * 100 + row['Date'].day

    
    # Filter the DataFrame for matches between the specified team and opponent from the same season, excluding the current match
    filtered_matches = df[(df[team_column] == row[team_column]) & 
                          (df[opponent_column] == row[opponent_column]) &
                          (df['Date_temp'] < row_date_temp)]
    
    recent_matches = filtered_matches.sort_values(by='Date', ascending=False).head(5)
    
    # Calculate weights - newer matches have higher weights
    weights = range(len(recent_matches), 0, -1)  # Descending list based on the number of matches
    
    # Calculate score based on the match result
    for match, weight in zip(recent_matches.itertuples(), weights):
        if getattr(match, 'FTR') == 'H' and getattr(match, team_column) == getattr(match, 'Team_ID') or \
           getattr(match, 'FTR') == 'A' and getattr(match, team_column) != getattr(match, 'Team_ID'):
            weighted_score += 3 * weight  # Team won
        elif getattr(match, 'FTR') == 'A':
            weighted_score += 1 * weight  # Draw
        
    # Normalize the weighted score by the sum of weights
    normalized_weighted_score = weighted_score / sum(weights) if weights else 0

    #print(f"Weighted score: {weighted_score}, Normalized weighted score: {normalized_weighted_score}")

    return normalized_weighted_score

In [245]:
df.tail(20)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Team_ID,Opp_ID,Ref_ID,Date_temp
3168,9,2024-04-07,1730,Tenerife,Eibar,2.0,1.0,H,1.0,1.0,...,2.08,1.83,2.08,1.88,2.02,1.80,220,70,54,20240407
312,2,2024-04-07,1800,Tottenham,Nott'm Forest,3.0,1.0,H,1.0,1.0,...,1.90,2.04,1.97,2.04,1.89,1.96,223,161,9,20240407
1557,4,2024-04-07,1945,Nantes,Lyon,1.0,3.0,A,1.0,0.0,...,2.00,1.93,2.01,1.93,1.96,1.88,154,135,54,20240407
2175,6,2024-04-07,1945,Juventus,Fiorentina,1.0,0.0,H,1.0,0.0,...,2.02,1.90,2.05,1.94,1.98,1.87,111,83,54,20240407
3169,9,2024-04-07,2000,Cartagena,Valladolid,0.0,2.0,A,0.0,2.0,...,1.91,2.00,1.93,2.03,1.86,1.95,46,231,54,20240407
3170,9,2024-04-08,1930,Santander,Alcorcon,0.0,1.0,A,0.0,0.0,...,1.99,1.92,2.04,1.93,1.93,1.88,199,3,54,20240408
1867,5,2024-04-08,1945,Auxerre,Troyes,2.0,0.0,H,0.0,0.0,...,1.97,1.92,1.99,1.96,1.92,1.87,18,226,54,20240408
2176,6,2024-04-08,1945,Udinese,Inter,1.0,2.0,A,1.0,0.0,...,1.93,2.02,1.95,2.02,1.89,1.95,227,108,54,20240408
18593,3,2024-04-09,1945,Millwall,Leicester,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,147,124,54,20240409
18594,3,2024-04-09,1945,Plymouth,QPR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,178,182,54,20240409


In [246]:
# Apply the modified function to create new columns
df['team_hist_vs'] = df.apply(lambda x: history_vs_opponent_weighted(df, x, 'Team_ID'), axis=1)
#df['opp_hist_vs'] = df.apply(lambda x: history_vs_opponent_weighted(df, x, 'Opp_ID'), axis=1)

In [247]:
import pandas as pd

# Function adapted for DataFrame application
def convert_odds(row):
    odds_win, odds_draw, odds_lose = row['AvgH'], row['AvgD'], row['AvgA']
    prob_win = 1 / odds_win
    prob_draw = 1 / odds_draw
    prob_lose = 1 / odds_lose
    prob_not_win = prob_draw + prob_lose
    return pd.Series([prob_win, prob_not_win], index=['probs_win', 'probs_not_win'])

# Apply the function and create new columns
#df[['probs_win', 'probs_not_win']] = df.apply(convert_odds, axis=1)

#df = df.drop(columns=['AvgH', 'AvgD', 'AvgA'])

In [248]:
def team_form(df, row, perspective):
    # Determine the team ID based on the perspective ('Team' or 'Opp')
    if perspective == 'Team':
        team_id = row['Team_ID']
    elif perspective == 'Opp':
        team_id = row['Opp_ID']
    else:
        raise ValueError("Perspective must be 'Team' or 'Opp'")
    
    # Get the current match date
    current_date = row['Date_temp']
    
    # Filter past matches for the team
    past_matches = df[((df['Team_ID'] == team_id) | (df['Opp_ID'] == team_id)) &
                      (df['Date_temp'] < current_date)].sort_values(by='Date_temp', ascending=False).head(5)
    
    # Initialize points
    points = 0
    
    # Weights for the matches (most recent match has the highest weight)
    weights = [5, 4, 3, 2, 1]
    
    # Calculate points with weights
    weighted_points_sum = 0
    total_weights = sum(weights[:len(past_matches)])  # Adjust the total weight in case of less than 5 matches
    
    for match, weight in zip(past_matches.itertuples(), weights):
        if (match.Team_ID == team_id and match.FTR == 'H') or (match.Opp_ID == team_id and match.FTR == 'A'):
            points += 3
        elif match.FTR == 'D':
            points += 1
        else:
            points += 0

        weighted_points_sum += points * weight
    
    if total_weights > 0:
        return weighted_points_sum / total_weights
    else:
        return 0  # Return 0 if no past matches found

In [249]:
df.tail(20)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Team_ID,Opp_ID,Ref_ID,Date_temp,team_hist_vs
3168,9,2024-04-07,1730,Tenerife,Eibar,2.0,1.0,H,1.0,1.0,...,1.83,2.08,1.88,2.02,1.80,220,70,54,20240407,1.000000
312,2,2024-04-07,1800,Tottenham,Nott'm Forest,3.0,1.0,H,1.0,1.0,...,2.04,1.97,2.04,1.89,1.96,223,161,9,20240407,3.000000
1557,4,2024-04-07,1945,Nantes,Lyon,1.0,3.0,A,1.0,0.0,...,1.93,2.01,1.93,1.96,1.88,154,135,54,20240407,0.500000
2175,6,2024-04-07,1945,Juventus,Fiorentina,1.0,0.0,H,1.0,0.0,...,1.90,2.05,1.94,1.98,1.87,111,83,54,20240407,2.600000
3169,9,2024-04-07,2000,Cartagena,Valladolid,0.0,2.0,A,0.0,2.0,...,2.00,1.93,2.03,1.86,1.95,46,231,54,20240407,1.000000
3170,9,2024-04-08,1930,Santander,Alcorcon,0.0,1.0,A,0.0,0.0,...,1.92,2.04,1.93,1.93,1.88,199,3,54,20240408,0.000000
1867,5,2024-04-08,1945,Auxerre,Troyes,2.0,0.0,H,0.0,0.0,...,1.92,1.99,1.96,1.92,1.87,18,226,54,20240408,2.666667
2176,6,2024-04-08,1945,Udinese,Inter,1.0,2.0,A,1.0,0.0,...,2.02,1.95,2.02,1.89,1.95,227,108,54,20240408,1.600000
18593,3,2024-04-09,1945,Millwall,Leicester,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,147,124,54,20240409,0.000000
18594,3,2024-04-09,1945,Plymouth,QPR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,178,182,54,20240409,0.000000


In [250]:
# Applying the function to each row for 'Team'
#df['team_form_team'] = df.apply(lambda row: team_form(df, row, 'Team'), axis=1)

# Applying the function to each row for 'Opp'
#df['team_form_opp'] = df.apply(lambda row: team_form(df, row, 'Opp'), axis=1)

In [251]:
def rolling_avgs(df, row, perspective, home_column, away_column):
    # Determine the team ID based on the perspective ('Team' or 'Opp')
    if perspective == 'Team':
        team_id = row['Team_ID']
    elif perspective == 'Opp':
        team_id = row['Opp_ID']
    else:
        raise ValueError("Perspective must be 'Team' or 'Opp'")
    
    # Get the current match date
    current_date = row['Date_temp']
    
    # Filter past 5 matches for the team
    past_matches = df[((df['Team_ID'] == team_id) | (df['Opp_ID'] == team_id)) &
                      (df['Date_temp'] < current_date)].sort_values(by='Date_temp', ascending=False).head(5)
    
    # Weights for the matches (most recent match has the highest weight)
    weights = [5, 4, 3, 2, 1]
    values = []
    
    # Determine which column to use and collect the values
    for match in past_matches.itertuples():
        if match.Team_ID == team_id:
            values.append(getattr(match, home_column))  # Use home_column for home team
        else:
            values.append(getattr(match, away_column))  # Use away_column for away team
    
    # Calculate the weighted average of the values
    weighted_sum = sum(value * weight for value, weight in zip(values, weights))
    total_weights = sum(weights[:len(values)])  # Adjust total weight if there are less than 5 matches
    
    if total_weights > 0:
        return weighted_sum / total_weights
    else:
        return 0  # Return 0 if no past matches found


In [252]:
#df['team_shots'] = df.apply(lambda row: rolling_avgs(df, row, 'Team', 'HS', 'AS'), axis=1)
#df['opp_shots'] = df.apply(lambda row: rolling_avgs(df, row, 'Opp', 'HS', 'AS'), axis=1)

#df['team_shots_target'] = df.apply(lambda row: rolling_avgs(df, row, 'Team', 'HST', 'AST'), axis=1)
#df['opp_shots_target'] = df.apply(lambda row: rolling_avgs(df, row, 'Opp', 'HST', 'AST'), axis=1)

In [253]:
from datetime import timedelta

def avg_games_played(df, row, team_column):
    team = row[team_column]
    # Ensure current_match_date is a Timestamp for comparison
    current_match_date = pd.to_datetime(row['Date'], dayfirst=True)  # Assuming 'Date' format is 'dd/mm/yy'

    delta = 30
    start_date = current_match_date - timedelta(days=delta)

    # Ensure 'Date' column is in datetime format for comparison
    #df['Date_temp'] = pd.to_datetime(df['Date'], dayfirst=True)  # Convert 'Date' column to datetime if not already done

    # Filter the DataFrame for matches within the last 30 days
    if team_column == 'Team_ID':
        past_matches = df[((df[team_column] == team) | (df['Opp_ID'] == team)) &
                          (df['Date'] >= start_date) & (df['Date'] <= current_match_date)]
    else:
        past_matches = df[((df['Team_ID'] == team) | (df[team_column] == team)) &
                          (df['Date'] >= start_date) & (df['Date'] <= current_match_date)]

    # If no matches were played in the last 30 days
    if past_matches.empty:
        return 0

    # Calculate weights based on the recency of each match
    weights = (current_match_date - past_matches['Date']).dt.days
    weighted_count = sum(delta - weights + 1)  # '+ 1' to include the match day in the weight

    # Normalize weights to sum to 1 and calculate the weighted average
    total_weight = sum(delta - weights + 1)
    weighted_avg = weighted_count / total_weight

    return weighted_avg


In [254]:
# Apply the function for each team and opponent
#df['team_avg_games'] = df.apply(lambda x: avg_games_played(df, x, 'Team_ID'), axis=1)
#df['opp_avg_games'] = df.apply(lambda x: avg_games_played(df, x, 'Opp_ID'), axis=1)

In [255]:
df.tail(20)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Team_ID,Opp_ID,Ref_ID,Date_temp,team_hist_vs
3168,9,2024-04-07,1730,Tenerife,Eibar,2.0,1.0,H,1.0,1.0,...,1.83,2.08,1.88,2.02,1.80,220,70,54,20240407,1.000000
312,2,2024-04-07,1800,Tottenham,Nott'm Forest,3.0,1.0,H,1.0,1.0,...,2.04,1.97,2.04,1.89,1.96,223,161,9,20240407,3.000000
1557,4,2024-04-07,1945,Nantes,Lyon,1.0,3.0,A,1.0,0.0,...,1.93,2.01,1.93,1.96,1.88,154,135,54,20240407,0.500000
2175,6,2024-04-07,1945,Juventus,Fiorentina,1.0,0.0,H,1.0,0.0,...,1.90,2.05,1.94,1.98,1.87,111,83,54,20240407,2.600000
3169,9,2024-04-07,2000,Cartagena,Valladolid,0.0,2.0,A,0.0,2.0,...,2.00,1.93,2.03,1.86,1.95,46,231,54,20240407,1.000000
3170,9,2024-04-08,1930,Santander,Alcorcon,0.0,1.0,A,0.0,0.0,...,1.92,2.04,1.93,1.93,1.88,199,3,54,20240408,0.000000
1867,5,2024-04-08,1945,Auxerre,Troyes,2.0,0.0,H,0.0,0.0,...,1.92,1.99,1.96,1.92,1.87,18,226,54,20240408,2.666667
2176,6,2024-04-08,1945,Udinese,Inter,1.0,2.0,A,1.0,0.0,...,2.02,1.95,2.02,1.89,1.95,227,108,54,20240408,1.600000
18593,3,2024-04-09,1945,Millwall,Leicester,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,147,124,54,20240409,0.000000
18594,3,2024-04-09,1945,Plymouth,QPR,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,178,182,54,20240409,0.000000


In [256]:
# Set the FTR to 'X' where the value is currently NaN
df['FTR'] = df['FTR'].fillna('X')

In [257]:
# Calculate means only for numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
means = df[numeric_cols].mean()

# Fill missing values in numeric columns with their respective means
df[numeric_cols] = df[numeric_cols].fillna(means)

In [258]:
# Drop every row where 'FTR' is not 'H', 'D', or 'A'
df = df[df['FTR'].isin(['H', 'D', 'A', 'X'])]

# FTR2 to store the FTR as '0', '1', or '2' for future reference
df['FTR2'] = df['FTR'].map({'H': 1, 'D': 0, 'A': 2, 'X': 0}).astype(int)

# Map 'H', 'D', and 'A' to 1, 0, and 0 respectively
df['FTR'] = df['FTR'].map({'H': 1, 'D': 0, 'A': 0, 'X': 0}).astype(int)

In [259]:
df.tail(20)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Team_ID,Opp_ID,Ref_ID,Date_temp,team_hist_vs,FTR2
3168,9,2024-04-07,1730,Tenerife,Eibar,2.000000,1.000000,1,1.000000,1.000000,...,2.080000,1.880000,2.020000,1.800000,220,70,54,20240407,1.000000,1
312,2,2024-04-07,1800,Tottenham,Nott'm Forest,3.000000,1.000000,1,1.000000,1.000000,...,1.970000,2.040000,1.890000,1.960000,223,161,9,20240407,3.000000,1
1557,4,2024-04-07,1945,Nantes,Lyon,1.000000,3.000000,0,1.000000,0.000000,...,2.010000,1.930000,1.960000,1.880000,154,135,54,20240407,0.500000,2
2175,6,2024-04-07,1945,Juventus,Fiorentina,1.000000,0.000000,1,1.000000,0.000000,...,2.050000,1.940000,1.980000,1.870000,111,83,54,20240407,2.600000,1
3169,9,2024-04-07,2000,Cartagena,Valladolid,0.000000,2.000000,0,0.000000,2.000000,...,1.930000,2.030000,1.860000,1.950000,46,231,54,20240407,1.000000,2
3170,9,2024-04-08,1930,Santander,Alcorcon,0.000000,1.000000,0,0.000000,0.000000,...,2.040000,1.930000,1.930000,1.880000,199,3,54,20240408,0.000000,2
1867,5,2024-04-08,1945,Auxerre,Troyes,2.000000,0.000000,1,0.000000,0.000000,...,1.990000,1.960000,1.920000,1.870000,18,226,54,20240408,2.666667,1
2176,6,2024-04-08,1945,Udinese,Inter,1.000000,2.000000,0,1.000000,0.000000,...,1.950000,2.020000,1.890000,1.950000,227,108,54,20240408,1.600000,2
18593,3,2024-04-09,1945,Millwall,Leicester,1.445544,1.178078,0,0.644917,0.518989,...,2.004487,2.011792,1.924688,1.929286,147,124,54,20240409,0.000000,0
18594,3,2024-04-09,1945,Plymouth,QPR,1.445544,1.178078,0,0.644917,0.518989,...,2.004487,2.011792,1.924688,1.929286,178,182,54,20240409,0.000000,0


In [260]:
df = df[[
    
        'Div', 'Date_temp', 'Time', 'Team_ID', 'Opp_ID', 'Ref_ID', 'FTR', 'FTR2',
        'team_hist_vs', 
        #'opp_hist_vs',

        #'probs_win',         
        #'probs_not_win', 
        
        #'team_form_team', 
        #'team_form_opp',
         
        #'team_shots', 'opp_shots',
        #'team_shots_target', 'opp_shots_target',

        #'team_avg_games', 'opp_avg_games',

        'AvgH', 'AvgD', 'AvgA'
         
         
         ]]

In [261]:
# Rename 'Date_temp' to 'Date'
df.rename(columns={'Date_temp': 'Date'}, inplace=True)

In [262]:
# Save the df to a CSV file
df.to_csv(f'data/processed/processed_data_{content}.csv', index=False)

In [263]:
import winsound
frequency = 400  # Set Frequency To 2500 Hertz
duration = 200  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)